!pip install yeelight
!pip install SpeechRecognition
!pip install ipywidgets

*YeeArt*

---

Este projeto tem como objetivo a comunicação com a lâmpada inteligente Xiaomi Yeelight. Nele, encontrará recursos e presets para interagir e experimentar com as diversas cores e flows que a lâmpada pode oferecer. 

O modo 'Sequências de cores' possui diversos flows programáveis.
O 'Cores fixas' possui diversas cores estáticas, utilizando o método .set_rgb().

Já o 'Reconhecimento de voz' te permite controlar o estado da lâmpada através do microfone:
O comando 'Fiat Lux' faz com que a luz acenda, o 'Good Night', com que apague-a e o 'Toggle' com que inverta seu estado atual. 


In [1]:
from yeelight import *
from yeelight import transitions
from yeelight import Flow
from yeelight import Bulb
import yeelight as yl
import speech_recognition as sr
import ipywidgets as wdg
import math
import time
import colour
from colour import Color

In [2]:
#Verifica se há lâmpadas conectadas na rede para o funcionamento do App.
if len(yl.discover_bulbs()) > 0:
    print("Pronto para rodar o programa!")
else:
    print("Não há lâmpadas conectadas na rede...")

Pronto para rodar o programa!


In [3]:
dispositivos_na_rede = lambda : [Bulb(ip, effect='smooth', duration=999) for ip in [dispositivo['ip'] for dispositivo in yl.discover_bulbs()]]
class Menu():
    def __init__(self, flows, statics, *bulbs):
        self.bulbs = [bulb for bulb in bulbs]
        self.bulb = bulbs[0]
        self.flows = flows
        self.flows_icons = flows_icons
        self.statics = statics
        self.comandos_de_voz = {'fiat lux' : self.bulb.turn_on, 'good night' : self.bulb.turn_off, 'toggle' : self.bulb.toggle}
        self.botoes_flow = [wdg.Button(description=word, layout=wdg.Layout( width='auto' ), button_style='warning', icon=self.flows_icons[word]) for word in self.flows.keys()]
        self.botoes_cores = [wdg.Button(description='', tooltip=word, layout=wdg.Layout(width='50%', height='90%'), style= {'button_color': rgb2hex(*map(lambda x : int(x), corgb)), 'font_weight' : 'normal'}) for word, corgb in self.statics.items()]
        self.botao_voz = wdg.Button(
                                    #description='Controle de voz',
                                    disabled=False,
                                    button_style='warning',
                                    tooltip="Controle de voz\n----------------\nComandos:\n----------------\n'Fiat lux'\n'Toggle'\n'Good night'",
                                    icon='microphone',    # (FontAwesome names without the `fa-` prefix)
                                    layout=wdg.Layout(width='50%', height='80%')
                                    )
        self.botao_toggle = wdg.ToggleButton(
                                    value=False,
                                    #description='',
                                    disabled=False,
                                    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                                    tooltip='Toggle',
                                    #border = 'dashed',
                                    icon='power-off',      # (FontAwesome names without the `fa-` prefix)
                                    layout=wdg.Layout(width='50%', height='90%'))
        self.botao_renomear = wdg.Button(
                                    description='Renomear dispositivo',
                                    disabled=False,
                                    button_style='info',
                                    tooltip='Mude este nome!',
                                    icon='',
                                    layout=wdg.Layout(width='50%', height='90%')
                                    )
        self.slider = wdg.IntSlider(min=1, max=100, value=50, description='', continuous_update=False,
                                    orientation='vertical',
                                    readout=True)
        self.slider.style.handle_color = 'red'
        
        self.box_cores = wdg.HBox(self.botoes_cores)
        self.box_header = wdg.VBox([wdg.HBox([self.botao_toggle, self.botao_voz, self.botao_renomear]), self.box_cores])
        self.box_flow = wdg.Box(children=self.botoes_flow, layout= wdg.Layout(#display='flex',
                                                                                flex_flow='column',
                                                                                align_items='stretch',
                                                                                border='ridge',
                                                                                width='90%'))
        #self.box_cores = wdg.Box(children=self.botoes_cores, layout= wdg.Layout(#display='flex',
         #                                                                       flex_flow='column',
          #                                                                      align_items='stretch',
           #                                                                     border='ridge', #groove
            #                                                                    width='90%'
             #                                                                   ))
        self.paleta_cores = wdg.ColorPicker(
                                            concise=True,
                                            description='',
                                            value='#efefef',
                                            height='100%',
                                            width='75%'
                                            )
        # Atribui a cada botão uma ação
        self.slider.observe(self.slider_brilho_handler)
        self.botao_toggle.observe(self.menu_toggle_handler)
        self.botao_renomear.on_click(self.menu_renomear_handler)
        self.botao_voz.on_click(self.menu_voz_handler)
        self.observa_botoes(self.botoes_flow, self.menu_flow_handler)
        self.observa_botoes(self.botoes_cores, self.menu_cores_handler)
        self.paleta_cores.observe(self.menu_paleta_handler)
        # ===============================================================

    def observa_botoes(self, botoes, handler):
        for botao in botoes:
            botao.on_click(handler)
                                  
    def slider_brilho_handler(self, botao):
        if botao['name'] == 'value':
            if botao['new'] <= 100:
                for bulb in self.bulbs:
                    bulb.set_brightness(botao['new'])
        
    def menu_toggle_handler(self, botao):
        for bulb in self.bulbs:
            bulb.toggle()
        
    def menu_cores_handler(self, botao):
        try:
            for bulb in self.bulbs:
                bulb.set_rgb(*statics[botao.tooltip])
        except:
            print("A conexão com a lâmpada foi encerrada.\nReconectando...")
            
    def menu_flow_handler(self, botao):
        flow = self.flows[botao.description]
        try:
            for bulb in self.bulbs:
                bulb.start_flow(flow)
        except:
            print("A conexão com a lâmpada foi encerrada.\nReconectando...")
                
    def menu_voz_handler(self, comandos):
        resultado = reconhece_voz()
        if resultado != -1:                                 # Se não houve nenhuma exceção
            if resultado in self.comandos_de_voz:
                self.comandos_de_voz[resultado]()
            else:
                print("Comando não reconhecido.")
    
    def menu_renomear_handler(self, botao):
        novo_nome = wdg.Textarea(
                                value=self.bulbs[0].get_properties()['name'],
                                placeholder='Escolha um novo nome para o seu dispositivo!',
                                description='',
                                disabled=False,
                                continuous_update=False
                                )
        display(novo_nome)
        def value_changed(change):
            if self.bulbs[0].get_properties()['name'] is not None:
                if self.bulbs[0].get_properties()['name'].lower() != change.new.lower():
                    self.bulbs[0].set_name(change.new)
                    print("Dispositivo renomeado com sucesso.")
                else: print("Entre com um nome diferente de {}".format(change.new))
            else: 
                try:
                    self.bulb.set_name(change.new)
                    print("Dispositivo renomeado com sucesso.")
                except:
                    print("Falha ao renomear o dispositivo.")
            
        novo_nome.observe(value_changed, 'value')

    def menu_paleta_handler(self, botao):
        if botao['name'] == 'value':
            cor_atual_rgb = tuple(int(botao['new'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
            #cor_atual_rgb = colour.hex2rgb(botao['new'])
            #print(botao['new'])
            #print(cor_atual_rgb)
            for bulb in self.bulbs:
                bulb.set_rgb(*cor_atual_rgb)
            
    def piano_keyboard(self):
        chunks = range(0,len(self.botoes_cores),7)
        black_tile = wdg.Button(description='', tooltip='black', layout=wdg.Layout(width='1%', height='15%'), style= {'button_color': '#000000', 'font_weight' : 'normal'})
        piano_keyboard = []
        for i in chunks:                                     
            chunk = self.botoes_cores[i:i+7]                 # Separa grupos de oitavas
            for tile in chunk:
                if (chunk.index(tile) + 1) not in [3, 7]:    # Acidentes semitonais
                    piano_keyboard.append(tile)
                    piano_keyboard.append(black_tile)
                else:
                    piano_keyboard.append(tile)
        return piano_keyboard

class App():
    def __init__(self, menu, header_button=None, left_button=None, center_button=None, right_button=None, footer_button=None):
        self.menu = menu
        self.header_button = header_button
        self.left_button = left_button
        self.center_button = center_button
        self.right_button = right_button
        self.footer_button = footer_button
        
    def layout(self):
        return wdg.AppLayout(   header=self.header_button,
                                left_sidebar=self.left_button,
                                center=self.center_button,
                                right_sidebar=self.right_button,
                                footer=self.footer_button,
                                #height="200px", width="50%",
                                grid_gap='45px',
                                justify_items='center', 
                                align_items='center', # 'flex-start', 'baseline', 'stretch'
                                pane_widths=[1, 1, 1],
                                pane_heights=[1, 5, 1] #[1, 5, '60px']
                            )
################################################################################ Funcionalidades:
def rgb2hex(r,g,b):
    return "#{:02x}{:02x}{:02x}".format(r,g,b)

def escolhe_flow(preset):    
    flow = Flow(count=0,                                                        #Roda flow indefinidamente (até comando stop_flow() ou turn_off())
            transitions=preset)
    return flow
        
def escada(span, teto):                                                         #Utilizar span máximo de 4*teto
    escadaria = list()
    for i in range(span):
        if i < teto:
            escadaria.append(i%teto+1)
        if i > teto and i < 2*teto:
            escadaria.append(((teto-1)-i)%teto+1)
        if i > 2*teto and i < 3*teto:
            escadaria.append(i%teto+1)
        if i > 3*teto and i < 4*teto:
            escadaria.append(((3*teto-1)-i)%teto+1)        
    return escadaria

def reconhece_voz():
    r = sr.Recognizer()
    mic = sr.Microphone()
    try:
        with mic as source:
            print("Iniciando reconhecimento...")
            r.adjust_for_ambient_noise(source)
            time.sleep(.250)
            print("Fale!")
            audio = r.listen(source)
            transcript = r.recognize_google(audio)
            return transcript.lower()
    except sr.UnknownValueError:
        print("Falha no reconhecimento de voz.")
        return -1
    except sr.RequestError:
        print("API fora do ar.")
        return -1

def main():
    dispositivos = dispositivos_na_rede()
    if len(dispositivos) > 1:
        menu_master = Menu(flows, statics, *dispositivos)
        menus_individuais = [Menu(flows, statics, dispositivo) for dispositivo in dispositivos]
        menus = (*menus_individuais, menu_master)
    else:
        menus = [Menu(flows, statics, dispositivo) for dispositivo in dispositivos]
    apps = [App(menu,header_button=menu.box_header,
                     left_button=menu.box_flow,
                     center_button=menu.slider,
                     right_button=menu.paleta_cores
                     #footer_button=menu.box_cores
               )
            for menu in menus]
    tabs = wdg.Tab()
    children = [app.layout() for app in apps]
    tabs.children = children
    for i in range(len(dispositivos)):
        tabs.set_title(i, dispositivos[i].get_properties()['name'])
    tabs.set_title(len(dispositivos), "Master")
    display(tabs)
    return apps, menus

################################################################################ Presets

def police3(duration=200, brightness=70):
    transitions = [
        RGBTransition(255, 0, 0, brightness=brightness, duration=duration),
        RGBTransition(200, 0, 0, brightness=1, duration=duration),
        RGBTransition(145, 0, 0, brightness=brightness, duration=duration),
        SleepTransition(duration=duration),
        RGBTransition(0, 0, 255, brightness=brightness, duration=duration),
        RGBTransition(0, 0, 200, brightness=1, duration=duration),
        RGBTransition(0, 0, 145, brightness=brightness, duration=duration),
        SleepTransition(duration=duration),
                   ]
    return transitions

def tranquilao(duration_light=5000, duration_sleep=500, brightness=49):
    transitions = [
        RGBTransition(255, 233, 207, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(201, 170, 136, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(74, 112, 139, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(226, 231, 186, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(255, 0, 38, brightness=brightness, duration=duration_light)
                   ]
    return transitions 

def psyched(duration_light=1229, duration_sleep=4500, brightness=27):
    transitions = [
        RGBTransition(255, 94,  27, brightness=brightness, duration=duration_light),
        RGBTransition(107, 255, 107, brightness=27, duration=690),
        RGBTransition(27,  151, 255, brightness=36, duration=duration_light),
        RGBTransition(107, 255, 80, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(255, 52,  224, brightness=brightness, duration=1500),
        RGBTransition(175, 119, 255, brightness=36, duration=duration_light),
        RGBTransition(228, 255, 163, brightness=27, duration=1000),
        RGBTransition(255, 0, 0, brightness=brightness, duration=2000)
                   ]
    return transitions 

def crazy_train(hue_max=359, saturation=100, duration=500):
    return [HSVTransition(hue, saturation=saturation, duration=duration) for hue in range(0, hue_max, math.ceil(hue_max/9))]

def come_down(duration):
    return [RGBTransition(0,195,111, brightness=1, duration=550),
            RGBTransition(0,180,132, brightness=1, duration=duration)]

def mobbin(duration_light=8500, duration_sleep=2300, brightness=12):
    return [RGBTransition(118, 52, 37, brightness=brightness, duration=duration_light),
            SleepTransition(duration=duration_sleep),
            RGBTransition(146, 83, 13, brightness=6, duration=duration_light),
            SleepTransition(duration=duration_sleep),
            RGBTransition(192, 106, 106, brightness=brightness, duration=duration_light),
            SleepTransition(duration=duration_sleep),
            RGBTransition(213, 33, 142, brightness=brightness, duration=duration_light),
            SleepTransition(duration=duration_sleep)]

def RedRedWine(brightness=100, duration_light=1000):
    return [RGBTransition(210, 0, 2, brightness=brightness, duration=440),
            RGBTransition(243, 0, 2, brightness=brightness, duration=440),
            RGBTransition(254, 2, 0, brightness=brightness, duration=duration_light),
            RGBTransition(233, 2, 0, brightness=69, duration=duration_light),
            RGBTransition(209, 0, 2, brightness=69, duration=duration_light),
            RGBTransition(148, 0, 2, brightness=40, duration=duration_light),
            RGBTransition(111, 0, 0, brightness=10, duration=duration_light),
            RGBTransition(52, 0, 0, brightness=5, duration=440),
            RGBTransition(42, 0, 0, brightness=brightness, duration=440)]

def Meditation(brightness=11, duration_light=10000):
    return [RGBTransition(122, 129, 255, brightness=brightness, duration=5000),
            RGBTransition(0, 255, 255, brightness=brightness, duration=4000),
            RGBTransition(0, 144, 81, brightness=brightness, duration=duration_light),
            RGBTransition(122, 129, 255, brightness=5, duration=duration_light),
            RGBTransition(0, 255, 255, brightness=4, duration=duration_light),
            RGBTransition(0, 144, 81, brightness=3, duration=duration_light),
            RGBTransition(77, 255, 41, brightness=1, duration=duration_light),
            RGBTransition(22, 0, 12, brightness=3, duration=4400),
            RGBTransition(0, 25, 255, brightness=brightness, duration=5500)]

################################################################################ Variáveis
flows_icons = { 
                'Desvio' : "500px",
                'Pleasergy' : "flask",
                'Politely' : "bullhorn",
                'Politicly' : "hiking",
                'Correct' : "",
                'Christ+' : "",
                'Impulse Train' : "",
                'RandInt' : "",
                'FlashMonO' : "",
                'FlashMulTi' : "microchip",
                'Wecker' : "",
                'Disco' : "",
                'Chill' : "umbrella",
                'Psychic' : "robot",
                'Slowlow' : "",
                'Latenight' : "database",
                'Wanderer' : "youtube",
                'Carménère Delight' : "",
                'Ommmmm' : "road"
               }
flows       = {
                'Desvio' : crazy_train(hue_max=342, saturation=94, duration=640),
                'Pleasergy' : yl.transitions.lsd(duration=2500, brightness=33),
                'Politely' : yl.transitions.police(duration=285, brightness=88),
                'Politicly' : yl.transitions.police2(duration=300, brightness=74),
                'Correct' : police3(duration=200, brightness=33), 
                'Christ+' : yl.transitions.christmas(duration=320, brightness=30, sleep=555),
                'Impulse Train' : yl.transitions.pulse(236, 195, 39, duration=400, brightness=77),
                'RandInt' : yl.transitions.randomloop(duration=750, brightness=100, count=9),
                'FlashMonO' : yl.transitions.strobe(), 
                'FlashMulTi' : yl.transitions.strobe_color(brightness=30),
                'Wecker' : yl.transitions.alarm(duration=300),
                'Disco' : yl.transitions.disco(bpm=25),
                'Chill' : tranquilao(duration_light=6000, duration_sleep=600, brightness=22),
                'Psychic' : psyched(duration_light=1445, duration_sleep=5000, brightness=17),
                'Slowlow' : yl.transitions.slowdown(duration=1000, brightness=80, count=8),
                'Latenight' : come_down(duration=7200),
                'Wanderer' : mobbin(duration_light=9000, duration_sleep=2500, brightness=50),
                'Carménère Delight' : RedRedWine(brightness=56, duration_light=300),
                'Ommmmm' : Meditation(brightness=11, duration_light=10000)
                }

flows       =  {mode : escolhe_flow(flow) for mode, flow in flows.items()}

statics     =  {
                'Vermelho' :        Color('red').get_rgb(),
                'Verde' :           Color('green').get_rgb(),
                'Azul' :            Color('blue').get_rgb(),
                'Magenta' :         Color('magenta').get_rgb(),
                'Ciano' :           Color('cyan').get_rgb(),
                'Amarelo' :         (255, 255, 0),
                'Camelo' :          (193, 138, 61),
                'Caiena' :          (148, 17, 0),
                'Verde-musgo' :     (0, 144, 81),
                'Roxo-nave' :       (83, 27, 147),
                'Azul-orquídea' :   (122, 129, 255),
                'Roxo-uva' :        (148, 55, 255),
                'Tangerina' :       (255, 147, 0),
                'Aqua' :            (0, 150, 255),
                'Tierra' :          (153, 102, 0),
                'Vitória' :         (0,   135, 223),
                'Derrota' :         (223, 135,   0)
                }

statics     =  {nome : tuple(map(lambda x: math.ceil(x * 255) if type(x) != int else x, cor)) for nome, cor in statics.items()}

In [4]:
apps, menus = main()

## -----------------------------------------TESTING AREA-----------------------------------------##

In [5]:
wdg.HBox(apps[0].menu.piano_keyboard())

In [6]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [7]:
#!jt -t monokai -f fira -fs 13 -nf ptsans -nfs 11 -N -kl -cursw 5 -cursc r -cellw 95% -T